# Introduction to the Low-salt-rejection Reverse Osmosis (LSRRO) Flowsheet
This tutorial shows how to build, initialize, simulate, and optimize an LSRRO flowsheet using WaterTAP.

![alt text](../docs/_static/flowsheets/lsrro.png)

## High-level Preview: Run an LSRRO optimization case.
#### 3-stage system
#### 70 g/L TDS feed, 50% Recovery


In [1]:
from watertap.flowsheets.lsrro.lsrro import (
    ACase,
    BCase,
    ABTradeoff,
    run_lsrro_case,
)
import logging
logging.getLogger("idaes").setLevel(logging.ERROR)
logging.getLogger("pyomo").setLevel(logging.ERROR)

In [2]:
# %%capture 
m, results = run_lsrro_case(
    number_of_stages=3,
    water_recovery=0.50,
    Cin=70,  # inlet NaCl conc kg/m3,
    Qin=1e-3,  # inlet feed flowrate m3/s
    A_case=ACase.optimize,
    B_case=BCase.optimize,
    AB_tradeoff=ABTradeoff.equality_constraint,
    has_NaCl_solubility_limit=True,
    has_calculated_concentration_polarization=True,
    has_calculated_ro_pressure_drop=True,
    permeate_quality_limit=500e-6,
    AB_gamma_factor=1,
    B_max=3.5e-6,
    number_of_RO_finite_elements=10,
    skip_initialization=True
)


Feed Concentration = 70.0 ppt
The current configuration is infeasible. Please adjust the decision variables.

***---Simulation results---***
Simulation failed. The current configuration is infeasible. Please adjust the decision variables.

***---Optimization results---***
----system metrics----
Feed: 1.05 kg/s, 66946 ppm
Product: 0.498 kg/s, 500 ppm
Brine: 0.548 kg/s, 127301 ppm
Volumetric water recovery: 50.0%
Number of Stages: 3
Total Membrane Area: 264.66
Energy Consumption: 8.8 kWh/m3
Levelized cost of water: 1.95 $/m3
Primary Pump Capital Cost ($/m3):0.5469168349767148
Booster Pump Capital Cost ($/m3): 0.1781879544453431
ERD Capital Cost ($/m3):0.044801159891790315
Membrane Capital Cost ($/m3): 0.16242752679371086
Indirect Capital Cost ($/m3): 0.4661667380537794
Electricity cost ($/m3): 0.6194467724701601


--decision variables--
Stage 1 operating pressure 85.0 bar
Stage 1 membrane area      85.0 m2
Stage 1 water perm. coeff.  1.5 LMH/bar
Stage 1 salt perm. coeff.  0.1 LMH
Stage 2

## Overview of setting up and optimizing the LSRRO flowsheet--step by step.
* Step 0: Import Python libraries.
* Step 1: Build the LSRRO flowsheet.
* Step 2: Set specifications for LSRRO flowsheet.
* Step 3: Initialize the LSRRO flowsheet.
* Step 4: Optimize the LSRRO flowsheet.
* Step 5: Visualize results.

## Step 0: Import libraries.

In [3]:
# Import LSRRO flowsheet module
from watertap.flowsheets.lsrro import lsrro

## Step 1: Build the LSRRO flowsheet.

In [4]:
# Create a Pyomo concrete model, flowsheet, and NaCl property parameter block.
m = lsrro.build(
    number_of_stages=3,
    has_NaCl_solubility_limit=True,
    has_calculated_concentration_polarization=True,
    has_calculated_ro_pressure_drop=True,
)


## Step 2: Set specifications for the LSRRO flowsheet.

In [5]:
lsrro.set_operating_conditions(m)

Feed Concentration = 70.0 ppt


## Step 3: Initialize the LSRRO flowsheet model.

In [6]:
lsrro.initialize(m)

--------------------START FORWARD INITIALIZATION PASS--------------------
--------------------START BACKWARD INITIALIZATION PASS--------------------
--------------------START FORWARD INITIALIZATION PASS--------------------
INFO: Starting Sequential Decomposition
INFO: Starting first pass run of network
2025-08-25 14:21:45 [INFO] idaes.init.fs.feed: Initialization Complete.
2025-08-25 14:21:46 [INFO] idaes.init.fs.PrimaryPumps[1].control_volume: Initialization Complete
2025-08-25 14:21:46 [INFO] idaes.init.fs.PrimaryPumps[1]: Initialization Complete: optimal - Optimal Solution Found
2025-08-25 14:21:47 [INFO] idaes.init.fs.Mixers[1]: Initialization Complete: optimal - Optimal Solution Found
2025-08-25 14:21:47 [INFO] idaes.init.fs.ROUnits[1].feed_side: Initialization Complete
2025-08-25 14:21:49 [INFO] idaes.init.fs.ROUnits[1]: Initialization Complete: optimal - Optimal Solution Found
2025-08-25 14:21:50 [INFO] idaes.init.fs.EnergyRecoveryDevices[1].control_volume: Initialization Comple

## Step 4: Simulate the LSRRO flowsheet.

In [7]:
lsrro.solve(m)


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 2039, 'Number of variables': 2039, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.22198796272277832}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Step 5: Optimize the LSRRO flowsheet.

In [8]:
lsrro.optimize_set_up(m,
    water_recovery=0.5,
    A_case=ACase.optimize,
    B_case=BCase.optimize,
    AB_tradeoff=ABTradeoff.equality_constraint,
    permeate_quality_limit=500e-6,
    )
optimization_results = lsrro.solve(m, raise_on_failure=True)

## 

## Step 6: Visualize results

In [ ]:
#TODO